# Import required dependencies

In [ ]:
from IPython.display import Audio
import matplotlib.pyplot as plt

# Download Youtube Video
Download youtube video using api/library

In [ ]:
import pytube
video_link = "https://www.youtube.com/watch?v=QTB1YiWxxKU"
video_file = pytube.YouTube(video_link)
stream = video_file.streams.first()
stream.download()

# Extract / Download Subtitle of Video
Extract Subtiles of Youtube Video and create a text file

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
subtitles = YouTubeTranscriptApi.get_transcript('QTB1YiWxxKU')
processed_transcript = ''
for sub in subtitles:
    processed_transcript += sub['text'] + ' '
print(processed_transcript)

# Create Inverted Index of a extracted transcript
After cleaning the subtitles create an inverted index and store in CSV File

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import itertools
import string

In [ ]:
def remove_stopword_punctuations(content):
    tokens = nltk.word_tokenize(content)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    filter_stopwords = [w for w in words if not w in stop_words]
    return filter_stopwords

def content_stemming(filtered_content):
    stemmed_content = []
    ps =  nltk.PorterStemmer()
    for token in filtered_content:
        word_stem = ps.stem(token)
        stemmed_content.append(word_stem.lower())
        
    return stemmed_content

In [ ]:
stopword_removal = remove_stopword_punctuations(processed_transcript)
stemmed_tokens = content_stemming(stopword_removal)

In [ ]:
import csv

with open('invertedindex.csv', mode='w') as invertedindex:
    employee_writer = csv.writer(invertedindex, delimiter=',')

    employee_writer.writerow(['Term', 'Frequency'])
    for token in stemmed_tokens:
        employee_writer.writerow([token, stemmed_tokens.count(token)])
    

In [ ]:
import pandas as pd
dataframe = pd.read_csv('invertedindex.csv')

In [ ]:
remove_repeated_terms = dataframe.drop_duplicates(subset=['Term'])

In [ ]:
invertedindex = remove_repeated_terms
invertedindex

# Extract Audio

In [ ]:
import moviepy.editor as mp
video_clip = mp.VideoFileClip(r"STOP WASTING TIME - Best Motivational Video.mp4")
video_clip.audio.write_audiofile("audiofile.wav")

# Seperate Voice and Music from Audio File

In [ ]:
Audio('audiofile.wav')

In [ ]:
!spleeter separate -o output/ audiofile.wav

In [ ]:
print('Voice in Video')
Audio('output/audiofile/vocals.wav')

In [ ]:
print('Music in Video')
Audio('output/audiofile/accompaniment.wav')

# Generate MFFC of Voice and Music File
Python Speech Library

In [ ]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import numpy as np

## Voice
Plot MFCC
Save MFFC of Voice in File

In [ ]:
(rate1,sig1) = wav.read("output/audiofile/vocals.wav")
mfcc_features_voice = mfcc(sig1,rate1)
f = open("mfcc_features_voice.txt", "w")
for row in mfcc_features_voice:
    np.savetxt(f, row)

f.close()

In [ ]:
plt.plot(mfcc_features_voice)
plt.show()

## Music
Plot MFFC
Save MFFC of Music in File

In [ ]:
(rate2,sig2) = wav.read("output/audiofile/accompaniment.wav")
mfcc_features_music = mfcc(sig2,rate2)
f = open("mfcc_features_music.txt", "w")
for row in mfcc_features_music:
    np.savetxt(f, row)

f.close()

In [ ]:
plt.plot(mfcc_features_music)
plt.show()

# MFCC using Librosa

In [ ]:
import librosa
import librosa.display

# Voice

In [ ]:
audio_path_vocals = 'output/audiofile/vocals.wav'
x, sr = librosa.load(audio_path_vocals)
print(type(x), type(sr))

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

# Music

In [ ]:
audio_path_vocals = 'output/audiofile/vocals.wav'
x, sr = librosa.load(audio_path_vocals)
print(type(x), type(sr))

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

# Extract Video Frames
Plot Histogram of Video Frames

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from FrameExtractor import FrameExtractor
%matplotlib inline

In [ ]:
FrameE = FrameExtractor('STOP WASTING TIME - Best Motivational Video.mp4')
FrameE.get_video_duration()
FrameE.get_n_images(every_x_frame=1000)
FrameE.extract_frames(every_x_frame=1000,img_name='kf',dest_path='key_images')
def show_image(path):
    image = cv2.imread(path)
    plt.imshow(image)
    plt.show()   
path, dirs, files = next(os.walk("key_images/"))
file_count = len(files)
print(file_count)
file=open("histogram.txt","w")
file.close()
for i in range(file_count):
    key_frame = 'key_images/kf_'+ str(i) +'.jpg'
    show_image(key_frame)
    im = cv2.imread(key_frame)
    # calculate mean value from RGB channels and flatten to 1D array
    vals = im.mean(axis=2).flatten()
    #print(vals)
    file=open("histogram.txt","a+")
    histogram = repr(vals)
    file.write(histogram)
    file.writelines("\n")   
    b, bins, patches = plt.hist(vals, 255)
    #print(bins)
    all_bins=repr(bins)
    file.write(all_bins)
    file.writelines("\n")
    plt.xlim([0,255])
    plt.show()
    #file.close

In [ ]:
#https://stackoverflow.com/questions/47899253/how-to-plot-3d-histogram-of-an-image-in-opencv
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import cv2

img = cv2.imread("key_images/kf_1.jpg")
img += cv2.imread("key_images/kf_2.jpg")
img += cv2.imread("key_images/kf_3.jpg")
img += cv2.imread("key_images/kf_4.jpg")
img += cv2.imread("key_images/kf_5.jpg")
img += cv2.imread("key_images/kf_6.jpg")
    

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv)
fig = plt.figure(figsize=(30, 40))

ax = fig.add_subplot(111, projection='3d')
for x, c, z in zip([h,s,v], ['r', 'g', 'b'], [30, 20, 10]):
    xs = np.arange(256)
    ys = cv2.calcHist([x], [0], None, [256], [0,256])
    cs = [c] * len(xs)
    cs[0] = 'c'
    ax.bar(xs, ys.ravel(), zs=z, zdir='y', color=cs, alpha=0.8)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()